In [1]:
#imported for system and file managing usage
import sys

import pandas as ps

#added to access the modules that placed in sub folder
sys.path.append('./modules')

#added to read special fucntions
from data import *

FUTURE_PERIOD_PREDICT = 1



In [2]:

filePath = '/home/cryptocurrency_Forcasting/dataset/Binance_BTCUSDT_1hour.csv'
x_train, y_train, x_validation, y_validation, x_test, y_test = readData(filePath, 20, 20, FUTURE_PERIOD_PREDICT)

36901       1502942400
36900       1502946000
36899       1502949600
36898       1502953200
36897       1502956800
             ...      
5        1626289200000
4        1626292800000
3        1626296400000
2        1626300000000
1        1626303600000
Name: unix, Length: 36901, dtype: int64
36901       1502942400
36900       1502946000
36899       1502949600
36898       1502953200
36897       1502956800
             ...      
5        1626289200000
4        1626292800000
3        1626296400000
2        1626300000000
1        1626303600000
Name: unix, Length: 36901, dtype: int64


36901    Thu Aug 17 04:00:00 2017
36900    Thu Aug 17 05:00:00 2017
36899    Thu Aug 17 06:00:00 2017
36898    Thu Aug 17 07:00:00 2017
36897    Thu Aug 17 08:00:00 2017
                   ...           
5        Wed Jul 14 19:00:00 2021
4        Wed Jul 14 20:00:00 2021
3        Wed Jul 14 21:00:00 2021
2        Wed Jul 14 22:00:00 2021
1        Wed Jul 14 23:00:00 2021
Name: unix, Length: 36901, dtype: object

                     unix                 date    symbol      open      high  \
36901 2017-08-17 04:00:00     2017-08-17 04-AM  BTC/USDT  16199.91  16199.91   
36900 2017-08-17 05:00:00     2017-08-17 05-AM  BTC/USDT   4308.83   4328.69   
36899 2017-08-17 06:00:00     2017-08-17 06-AM  BTC/USDT   4315.32   4345.45   
36898 2017-08-17 07:00:00     2017-08-17 07-AM  BTC/USDT   4324.35   4349.99   
36897 2017-08-17 08:00:00     2017-08-17 08-AM  BTC/USDT   4349.99   4377.85   
...                   ...                  ...       ...       ...       ...   
5     2021-07-14 19:00:00  2021-07-14 19:00:00  BTC/USDT  32743.96  32798.62   
4     2021-07-14 20:00:00  2021-07-14 20:00:00  BTC/USDT  32729.77  32869.58   
3     2021-07-14 21:00:00  2021-07-14 21:00:00  BTC/USDT  32800.41  32940.00   
2     2021-07-14 22:00:00  2021-07-14 22:00:00  BTC/USDT  32852.24  33063.82   
1     2021-07-14 23:00:00  2021-07-14 23:00:00  BTC/USDT  32959.97  32974.44   

            low     close   Volume BTC 

In [3]:
1502942400

1502942400

In [4]:
import time
temp=1502942400
time.ctime(int(temp))

'Thu Aug 17 04:00:00 2017'